In [4]:
import pandas as pd 
import numpy as np
from datetime import datetime
from pandas.io.json import json_normalize
import pickle

# Loading the test data set

- This Jupyter Notebook requires a test file in .csv format 

- There are 2 ways to test this file: 

- Uncomment the blocks of code for whichever method (METHOD 1 OR METHOD 2) you want to test the Random Forest model on.

METHOD 1:
- If you want to use a CSV of (longitude, latitude, timestamp) uncomment the block of code below.

- Pass the filename of the test CSV dataset in the placeholder for **path** variable

In [5]:
lat_long_filename = ""
weather_filename = ""

In [6]:
# #METHOD:1 Block of code

# path = '<filename>' #enter the path to your CSV test file here. (Recommended: copy the file in the current folder)

# long_lat_timestamp = pd.DataFrame()
# lat_long_timestamp = pd.DataFrame(columns=['Latitude', 'Longitude', 'Birth Date'])
# prefetch = False

# # files
# lat_long_filename = 'lat_long_timestamp_test.csv'
# weather_filename = 'weather_dataframe_test.csv'

# if(path == ''):
#     print("Please enter a valid file name")
# else:
#     long_lat_timestamp = pd.read_csv(path)
#     lat_long_timestamp['Latitude'] = long_lat_timestamp['Latitude']
#     lat_long_timestamp['Longitude'] = long_lat_timestamp['Longitude']
#     lat_long_timestamp['Birth Date'] = long_lat_timestamp['Birth Date']    

In [7]:
long_lat_timestamp = pd.DataFrame(columns=['Latitude', 'Longitude', 'Birth Date'])

METHOD 2:
- If you want to test against pre-fetched data, uncomment the 3 lines in the cell below 

In [8]:
# #METHOD:2 Block of code

# prefetch = True
# lat_long_filename = 'lat_long_timestamp_test_500.csv'
# weather_filename = 'weather_dataframe_test_500.csv'
        

# Function to label season based on timestamp (birth date)

- The function **get_season_from_month()** 

In [9]:
def get_season_from_month(timestamp):
    dateTimeObject = datetime.fromtimestamp(timestamp)
    month = dateTimeObject.month
    if(month in [1, 2, 12]):
        return "Winter"
    elif(month in [3, 4, 5]):
        return "Spring"
    elif(month in [6, 7, 8]):
        return "Summer"
    elif(month in [9, 10, 11]):
        return "Fall"

- Create an empty dataframe called "weather dataframe" to store the data fetched from the "Darksky" API

In [10]:
column_names = ['apparentTemperatureHigh', 'apparentTemperatureHighTime',
       'apparentTemperatureLow', 'apparentTemperatureLowTime',
       'apparentTemperatureMax', 'apparentTemperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime', 'cloudCover',
       'dewPoint', 'humidity', 'icon', 'moonPhase', 'precipAccumulation',
       'precipIntensity', 'precipIntensityMax', 'precipIntensityMaxTime',
       'precipProbability', 'precipType', 'pressure', 'summary', 'sunriseTime',
       'sunsetTime', 'temperatureHigh', 'temperatureHighTime',
       'temperatureLow', 'temperatureLowTime', 'temperatureMax',
       'temperatureMaxTime', 'temperatureMin', 'temperatureMinTime', 'time',
       'uvIndex', 'uvIndexTime', 'visibility', 'windBearing', 'windGust',
       'windGustTime', 'windSpeed']
weather_dataframe = pd.DataFrame(columns = column_names)

- Create an empty numpy array to accept empty responses from the Darksky API

In [11]:
nan_array = np.empty((1,len(column_names)))
nan_array[:] = np.nan

# Fetching weather features from API

- We are using the 3-tuple of (latitude, longitude, timestamp) from the test file to fetch data from the API.
- Using a for loop, we will pass each 3-tuple from our lat_long_timestamp dataframe to obtain weather data at that particular location and time (daily aggregated). 
- Each set of features obtained foe every 3-tuple will be added to our empty initialized dataframe called "weather_dataframe"

In [12]:
if(prefetch == False):
    lat_long_timestamp["datetime"] = lat_long_timestamp["Birth Date"].apply(lambda x: datetime.fromtimestamp(x))
    lat_long_timestamp["Season"] = lat_long_timestamp['Birth Date'].apply(lambda x: get_season_from_month(x) )


    lat_long_timestamp.to_csv(lat_long_filename)
    api_key = "81bbbccc7288b8cc8f5e3661483a86cd"

    for i in range(0,len(lat_long_timestamp)):
    #     print("https://api.darksky.net/forecast/"+api_key+"/"+str(lat_long_timestamp["Latitude"][i])+","+str(lat_long_timestamp["Longitude"][i])+","+str(lat_long_timestamp["Birth Date"][i])+"?exclude=currently,flags"+"\t"+str(i)+"\n")
        output = []
        df_b = []
        try:
            output = pd.read_json("https://api.darksky.net/forecast/"+api_key+"/"+str(lat_long_timestamp["Latitude"][i])+","+str(lat_long_timestamp["Longitude"][i])+","+str(lat_long_timestamp["Birth Date"][i])+"?exclude=currently,flags",orient='columns')
            df_b = pd.DataFrame.from_dict(json_normalize(output["daily"][0]),orient='columns')
        except:
            output = nan_array
            df_b = pd.DataFrame(nan_array,columns=column_names)
        weather_dataframe = pd.concat([weather_dataframe,df_b])

    weather_dataframe.to_csv(weather_filename)

# Data Preprocessing

- We will load(read) the written CSV files OR the pre-fetched dattsets to clean & pre-process the data

In [13]:
lat_long_timestamp = pd.read_csv(lat_long_filename)

In [14]:
weather_dataframe = pd.read_csv(weather_filename)

In [15]:
weather_dataframe.head()

,Unnamed: 0,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,...,temperatureMin,temperatureMinTime,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed
0,0,83.24,1.534455e+09,67.04,1.534500e+09,83.24,1.534455e+09,52.64,1.534407e+09,0.26,...,52.15,1.534407e+09,1.534392e+09,7.0,1.534442e+09,10.000,140.0,9.64,1.534439e+09,3.34
1,0,52.99,1.520721e+09,18.34,1.520773e+09,52.99,1.520721e+09,23.83,1.520672e+09,0.32,...,28.51,1.520669e+09,1.520662e+09,6.0,1.520707e+09,9.997,350.0,25.81,1.520744e+09,7.58
2,0,39.26,1.552857e+09,17.76,1.552895e+09,39.26,1.552857e+09,11.29,1.552826e+09,0.43,...,17.73,1.552830e+09,1.552802e+09,4.0,1.552851e+09,10.000,226.0,5.97,1.552820e+09,3.35
3,0,103.94,1.534114e+09,69.55,1.534159e+09,103.94,1.534114e+09,66.11,1.534073e+09,0.00,...,65.62,1.534073e+09,1.534054e+09,9.0,1.534100e+09,10.000,165.0,23.20,1.534054e+09,9.97
4,0,82.61,1.530122e+09,72.26,1.530183e+09,82.61,1.530122e+09,66.90,1.530090e+09,0.55,...,65.59,1.530090e+09,1.530072e+09,8.0,1.530122e+09,9.997,203.0,21.09,1.530133e+09,5.38


- Let us drop the first unnecessary column with all zero/index values from both the data frames

In [16]:
lat_long_timestamp = lat_long_timestamp.drop(lat_long_timestamp.columns[0], axis=1)
weather_dataframe = weather_dataframe.drop(weather_dataframe.columns[0], axis=1)

- Let us drop the rows in the weather dataframe that have NaN/null values in all the rows. 

In [17]:
weather_dataframe = weather_dataframe.dropna(how='all')

In [18]:
weather_dataframe = weather_dataframe.fillna(weather_dataframe.median())

- Let us join the lat_long_timestamp & the weather_dataframe together on index (Inner Join) to obtain a complete dataset including the 3-tuples (latitude, longitude, timestamp) & their corresponding weather features obtained from the API

In [19]:
final_df = pd.merge(lat_long_timestamp, weather_dataframe, left_index=True, right_index=True)

- Let's check the NaN values in each column. 

- NOTE: One better way to handle columns with a small number of NaN values is to impute them by one of the folowing ways:
       - Mean/Median/Mode
       - KNN Imputation
       - Predictive Imputation
- For the purposes of our dataset, we will impute the median value for columns that have any NaN values


In [20]:
# Features selected from the training set

features = ['apparentTemperatureHigh', 'apparentTemperatureLow',
       'apparentTemperatureMax', 'apparentTemperatureMin', 'dewPoint',
       'sunriseTime', 'temperatureHigh', 'temperatureLow', 'temperatureMax',
       'temperatureMin', 'time', 'uvIndex']

In [21]:
features.append("Season")

In [22]:
features

['apparentTemperatureHigh',
 'apparentTemperatureLow',
 'apparentTemperatureMax',
 'apparentTemperatureMin',
 'dewPoint',
 'sunriseTime',
 'temperatureHigh',
 'temperatureLow',
 'temperatureMax',
 'temperatureMin',
 'time',
 'uvIndex',
 'Season']

In [23]:
final_df_2 = final_df[features]

In [24]:
y_test = final_df_2["Season"]
x_test = final_df_2.loc[:, final_df_2.columns != 'Season']

In [25]:
x_test = x_test.fillna(x_test.median())

- Now, let us do preprocessing of our test dataset using the preprocessing module from the scikit-learn library.

In [26]:
from sklearn import preprocessing

#Scaling data..

scaler = preprocessing.MinMaxScaler()
scaler.fit(x_test)
x_test = pd.DataFrame(scaler.transform(x_test), index=x_test.index, columns=x_test.columns)


In [27]:
filename = 'RandomForestSeasonClassifier.sav'

RF_Classifier_model = pickle.load(open(filename, 'rb'))
result = RF_Classifier_model.score(x_test, y_test)

In [28]:
print("Random Classifier Accuracy Percentage: " +str(result*100) + "%")

Random Classifier Accuracy Percentage: 98.50746268656717%
